In [1]:
from datetime import datetime
import traceback
import functools
import multiprocessing
from multiprocessing import Pool

import time
import os
import sys
from pathlib import Path

import numpy as np
import scipy
import pandas as pd
import ipympl
import matplotlib
#matplotlib.use('nbagg')
from matplotlib import style
style.use('ggplot')
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

from astropy import stats

import astropy.units as u
from astropy.io import fits
from mmtwfs.wfs import WFSFactory

In [2]:
def get_traceback(f):
    @functools.wraps(f)
    def wrapper(*args, **kwargs):
        try:
            return f(*args, **kwargs)
        except Exception as ex:
            ret = '#' * 60
            ret += "\nException caught:"
            ret += "\n"+'-'*60
            ret += "\n" + traceback.format_exc()
            ret += "\n" + '-' * 60
            ret += "\n"+ "#" * 60
            print(sys.stderr, ret)
            sys.stderr.flush()
            raise ex
 
    return wrapper

In [3]:
# instantiate all of the WFS systems...
wfs_keys = ['newf9', 'f5', 'mmirs', 'binospec']
wfs_systems = {}
wfs_names = {}
for w in wfs_keys:
    wfs_systems[w] = WFSFactory(wfs=w)
    wfs_names[w] = wfs_systems[w].name
plt.close('all')

# it appears the binospec reference LED doesn't use a pinhole so the reference spots are smeared out. as a hack
# we'll assume the spots are nyquist sampled like the f5 and mmirs systems.
#wfs_systems['binospec'].modes['binospec']['reference']['fwhm'] = 2.0
#wfs_systems['binospec'].modes['binospec']['reference']['sigma'] = 2.0 * stats.funcs.gaussian_fwhm_to_sigma

# give mmirs a default
wfs_systems['mmirs'].default_mode = 'mmirs1'

/home/tim/miniconda3/lib/python3.6/site-packages/matplotlib/__init__.py:942: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)
/home/tim/miniconda3/lib/python3.6/site-packages/matplotlib/__init__.py:942: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)
/home/tim/miniconda3/lib/python3.6/site-packages/matplotlib/__init__.py:942: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)
/home/tim/miniconda3/lib/python3.6/site-packages/matplotlib/__init__.py:942: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)
/home/tim/miniconda3/lib/python3.6/site-packages/matplotlib/__init__.py:942: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


In [4]:
def seeing(df, wfs):
    """
    Given a sigma derived from a gaussian fit to a WFS spot, deconvolve the systematic width from the reference image
    and relate the remainder to r_0 and thus a seeing FWHM.
    """
    # the effective wavelength of the WFS imagers is about 600-650 nm. we use 650 nm to maintain consistency
    # with the value used by the old SHWFS system.
    wave = 650 * u.nm
    wave = wave.to(u.m).value  # r_0 equation expects meters so convert

    mode = wfs.default_mode
    
    # calculate the physical size of each aperture.
    ref = wfs.modes[mode]['reference']
    apsize_pix = np.max((ref['xspacing'], ref['yspacing']))
    d = wfs.telescope.diameter * apsize_pix / wfs.pup_size
    d = d.to(u.m).value  # r_0 equation expects meters so convert

    # we need to deconvolve the instrumental spot width from the measured one to get the portion of the width that
    # is due to spot motion
    ref_sigma = stats.funcs.gaussian_fwhm_to_sigma * ref['fwhm']
    sigma = stats.funcs.gaussian_fwhm_to_sigma * df['fwhm']
    
    corr_sigma = np.sqrt(sigma**2 - ref_sigma**2)
    corr_sigma *= wfs.pix_size.to(u.rad).value  # r_0 equation expects radians so convert

    # this equation relates the motion within a single aperture to the characteristic scale size of the
    # turbulence, r_0.
    r_0 = ( 0.179 * (wave**2) * (d**(-1/3))/corr_sigma**2 )**0.6

    # this equation relates the turbulence scale size to an expected image FWHM at the given wavelength.
    raw_seeing = u.Quantity(u.rad * 0.98 * wave / r_0, u.arcsec)

    # correct seeing to zenith
    seeing = raw_seeing / df['airmass']**0.6

    return raw_seeing, seeing

In [5]:
def check_wfs(p):
    """
    Check to see which system was used on a given night
    """
    if Path.exists(p / "F9"):  # wow, i really like pathlib.Path...
        return "newf9"
    elif Path.exists(p / "F5"):
        return "f5"
    elif Path.exists(p / "MMIRS"):
        return 'mmirs'
    else:
        return None

@get_traceback
def process_image(f, wfskey=None):
    """
    Process FITS file, f, to get info we want from the header and then analyse it with the 
    appropriate WFS instance. Return results in a comma-separated line that will be collected 
    and saved in a CSV file.
    """
    if "ave" not in f.name:
        with fits.open(f) as h:
            hdr = h[-1].header
            
            # if wfskey is None, figure out which WFS from the header info...
            if wfskey is None:
                # check for MMIRS
                if 'WFSNAME' in hdr:
                    if 'mmirs' in hdr['WFSNAME']:
                        wfskey = 'mmirs'
                if 'mmirs' in f.name:
                    wfskey = 'mmirs'
                    
                # check for binospec
                if 'bino' in f.name:
                    wfskey = 'binospec'
                if 'ORIGIN' in hdr:
                    if 'Binospec' in hdr['ORIGIN']:
                        wfskey = 'binospec'
                        
                # check for F/9
                if 'f9wfs' in f.name:
                    wfskey = 'newf9'
                if 'OBSERVER' in hdr:
                    if 'F/9 WFS' in hdr['OBSERVER']:
                        wfskey = 'newf9'
                if wfskey is None and 'CAMERA' in hdr:
                    if 'F/9 WFS' in hdr['CAMERA']:
                        wfskey = 'newf9'
                        
                # check for F/5 (hecto)
                if wfskey is None and 'SEC' in hdr:  # mmirs has SEC in header as well and is caught above
                    if 'F5' in hdr['SEC']:
                        wfskey = 'f5'
                        
            if wfskey is None:
                # if wfskey is still None at this point, bail out.
                print(f"Can't determine WFS for {f.name}...")
                return None
            # else:
                # print(f"Using {wfskey} for {f.name}...")

            if 'AIRMASS' in hdr:
                airmass = hdr['AIRMASS']
            else:
                airmass = np.nan

            # we need to fix the headers in all cases to have a proper DATE-OBS entry with
            # properly formatted FITS timestamp.  in the meantime, this hack gets us what we need 
            # for analysis in pandas.
            dtime = None
            if 'DATEOBS' in hdr:
                dateobs = hdr['DATEOBS']
                ut = hdr['ut']
                timestring = dateobs + " " + ut + " UTC"
                if wfskey in ('newf9', 'f5'):
                    dtime = datetime.strptime(timestring, "%a %b %d %Y %H:%M:%S %Z")
                else:
                    dtime = datetime.strptime(timestring, "%Y-%m-%d %H:%M:%S %Z")
            else:
                timestring = hdr['DATE-OBS'] + " UTC"
                try:
                    dtime = datetime.strptime(timestring, "%Y-%m-%dT%H:%M:%S.%f %Z")
                except:
                    dtime = datetime.strptime(timestring, "%Y-%m-%dT%H:%M:%S %Z")
                
            if dtime is None:
                print(f"No valid timestamp in header for {f.name}...")
                return None
            else:
                obstime = dtime.isoformat()
                
            # being conservative here and only using data that has proper slope determination
            # and wavefront solution. also want to get statistics on the quality of the wavefront fits.
            try:
                results = wfs_systems[wfskey].measure_slopes(str(f), plot=False)
            except:
                print(f"Problem analyzing {f.name}...")
                results = {}
                results['slopes'] = None

            if results['slopes'] is not None:
                zresults = wfs_systems[wfskey].fit_wavefront(results, plot=False)
                line = "%s,%s,%s,%f,%f,%f,%f,%f\n" % (
                    obstime,
                    wfskey,
                    f.name, 
                    airmass,
                    results['seeing'].value,
                    results['raw_seeing'].value,
                    results['fwhm'],
                    zresults['residual_rms'].value
                )
                zfile = f.parent / (f.stem + ".zernike")
                zresults['zernike'].save(filename=zfile)
                return line
            else:
                return None
    return None
#rootdir = Path("/Users/tim/MMT/wfsdat/test")
#rootdir = Path("/Volumes/LaCie/wfsdat")
#rootdir = Path("/Volumes/Seagate2TB/wfsdat")
rootdir = Path("/mnt/d/wfsdat")

In [6]:
dirs = sorted(list(rootdir.glob("201803*")))  # pathlib, where have you been all my life!
csv_header = "time,wfs,file,airmass,seeing,raw seeing,fwhm,wavefront rms\n"
for d in dirs:
    if d.is_dir():
        if Path.exists(d / "seeing_results.csv"):
            print("Already processed %s..." % d.name)
        else:
            try:
                lines = []
                lines.append(csv_header)
                night = int(d.name)  # valid WFS directories are ints of the form YYYYMMDD. if not this form, int barfs
                msg = "checking %d... " % night
                fitsfiles = d.glob("*.fits")
                print(msg)
#                 for f in fitsfiles:
#                     print("Processing %s..." % f)
#                     process_image(f)
                nproc = 8
                with Pool(processes=nproc) as pool:  # my mac's i7 has 4 cores + hyperthreading so 8 virtual cores. 
                    plines = pool.map(process_image, fitsfiles)  # plines comes out in same order as fitslines!
                plines = list(filter(None.__ne__, plines))  # trim out any None entries
                lines.extend(plines)
                with open(d / "seeing_results.csv", "w") as f:
                    f.writelines(lines)
            except ValueError as e:  # this means running int(d.name) failed so it's not a valid directory...
                print(f"Skipping %s... ({e})" % d.name)

Already processed 20180301...
Already processed 20180302...
Already processed 20180303...
Already processed 20180304...
Already processed 20180305...
Already processed 20180306...
Already processed 20180307...
Already processed 20180308...
Already processed 20180309...
Already processed 20180310...
Already processed 20180311...
Already processed 20180312...
Already processed 20180313...
Already processed 20180314...
Already processed 20180315...
Already processed 20180316...
Already processed 20180317...
Already processed 20180318...
Already processed 20180319...
Already processed 20180320...
Already processed 20180321...
Already processed 20180322...
checking 20180323... 
checking 20180324... 


Wavefront slope measurement failed: (WFSAnalysisFailed(...), 'Only 15 spots detected out of 151 apertures.')


Can't determine WFS for tmp.fits...
checking 20180325... 
checking 20180326... 
checking 20180327... 
Can't determine WFS for tmp.fits...
checking 20180328... 
Can't determine WFS for tmp.fits...
checking 20180329... 


Wavefront slope measurement failed: (WFSAnalysisFailed(...), 'Only 11 spots detected out of 151 apertures.')


Can't determine WFS for tmp.fits...
checking 20180330... 
Can't determine WFS for tmp.fits...
checking 20180331... 
Can't determine WFS for tmp.fits...


In [8]:
# use find to make this csv list
with open(rootdir / "csvs_2018q1.txt") as f:
    csvs = f.readlines()

# loop through the csv files and read them with pandas
frames = []
for csv in csvs:
    frames.append(pd.read_csv(rootdir / csv.rstrip()))

# now concat each frame into one big frame containing all of the data
data = pd.concat(frames)

In [15]:
g = data.groupby('wfs')
g.wfs.describe()

,count,unique,top,freq
wfs,,,,
binospec,2191,1,binospec,2191
f5,318,1,f5,318
newf9,607,1,newf9,607


In [30]:
# re-calculate the seeing using updated spot FWHM for binospec. filter out any NaN values...
#fixed_list = []
#for wfskey, group in g:
#    wfs = wfs_systems[wfskey]
#    group = g.get_group(wfskey)
#    #group.apply(seeing, axis=1, args=(wfs,))
#    group['fixed_raw_seeing'], group['fixed_seeing'] = seeing(group, wfs)
#    fixed_list.append(group)
#fixed = pd.concat(fixed_list)
#fixed.rename(index=str, columns={"seeing": "orig_seeing", "raw seeing": "orig_raw_seeing"}, inplace=True)
#fixed.rename(index=str, columns={"fixed_seeing": "seeing", "fixed_raw_seeing": "raw seeing"}, inplace=True)
fixed = data[np.isfinite(data['seeing'])]
fixed = fixed[fixed['fwhm'] > 0.]
fixed = fixed[fixed['seeing'] < 4.]

In [31]:
fixed.sort_values(by='seeing')

,time,wfs,file,airmass,seeing,raw seeing,fwhm,wavefront rms
7,2018-01-15T04:30:38.237000,newf9,f9wfs_20180114-213008.fits,1.2606,0.382894,0.439972,5.658443,190.808161
3,2018-01-14T01:43:16.984000,newf9,f9wfs_20180113-184255.fits,1.2091,0.413421,0.463308,5.809855,186.347547
6,2018-01-15T04:29:43.931000,newf9,f9wfs_20180114-212913.fits,1.2573,0.428167,0.491220,5.992432,187.253914
340,2018-02-09T11:12:39.530000,binospec,wfs_ff_cal_img_2018.0209.111239.fits,1.0117,0.430659,0.433675,3.264558,2827.921500
9,2018-02-05T06:05:01.526000,newf9,f9wfs_20180204-230454.fits,1.2918,0.453667,0.528998,6.241627,173.827947
4,2018-01-15T04:25:19.899000,newf9,f9wfs_20180114-212450.fits,1.2420,0.459607,0.523431,6.204772,146.559188
3,2018-01-23T04:09:22.689000,newf9,f9wfs_20180122-210925.fits,1.1274,0.476918,0.512496,6.132511,136.633181
13,2018-03-22T05:42:30,f5,test_0002.fits,1.1800,0.477580,0.527443,4.261168,190.268470
98,2018-01-22T10:13:36.321000,newf9,f9wfs_20180122-031340.fits,1.0882,0.497289,0.523159,6.202976,172.323778
8,2018-02-05T06:04:12.852000,newf9,f9wfs_20180204-230406.fits,1.2948,0.498896,0.582548,6.597987,163.451947


In [32]:
# create a date-time index so we can group and analyze the data by timestamps
fixed = fixed.set_index(pd.DatetimeIndex(fixed['time'], name='ut'))

In [33]:
# make raw histogram of the zenith-corrected seeing column
fixed.hist(column='seeing', bins=100, range=(0.0, 4.0), alpha=0.6)
plt.title("2018-01-01 through 2018-03-31")
plt.xlabel("Seeing (arcsec)")
plt.ylabel("N")
plt.show()

/home/tim/miniconda3/lib/python3.6/site-packages/matplotlib/__init__.py:942: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

In [34]:
# fit a log-normal distribution to the seeing data
sigma, loc, exp_mu = scipy.stats.lognorm.fit(fixed['seeing'])
print(sigma, loc, exp_mu)
x = np.arange(0.0, 4.0, 0.01)
p = scipy.stats.lognorm.pdf(x, sigma, loc=loc, scale=exp_mu)
mu = np.log(exp_mu)
mode = np.exp(mu - sigma**2) + loc
median = np.nanmedian(fixed['seeing'])
fit_median = exp_mu + loc

0.576297137102 0.330595619397 0.822444076502


In [35]:
# plot normalized histogram with the fit and the median/mode calculated from the fit.
plt.hist(fixed['seeing'], normed=True, bins=100, range=(0.0, 4.0), label="hist", alpha=0.6)
logp = plt.plot(x, p, label="logp")
plt.xlabel("Seeing (arcsec)")
plt.ylabel("Normalized N")
plt.title("2018-01-01 through 2018-03-31")
plt.legend(["median=%.2f\", mode=%.2f\"" % (fit_median, mode), "median=%.2f\"" % (
    median
)])
plt.savefig("2018q1_hist.pdf")
plt.show()

/home/tim/miniconda3/lib/python3.6/site-packages/matplotlib/__init__.py:942: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)
/home/tim/miniconda3/lib/python3.6/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


FigureCanvasNbAgg()

In [36]:
# panda time indexing makes it easy to group by month
months = {}
legends = []
months['January'] = fixed['2018-01']
months['February'] = fixed['2018-02']
months['March'] = fixed['2018-03']
for m in months:
    plt.hist(months[m]['seeing'], bins=100, range=(0.0, 4.0), label=m, alpha=0.6)
    legend = "%s: %.2f\"" % (m, np.median(months[m]['seeing']))
    legends.append(legend)
plt.legend(legends)
plt.xlabel("Seeing (arcsec)")
plt.ylabel("N")
#plt.title("Data and median seeing values grouped by month")
plt.savefig("2018q1_monthly.pdf")
plt.show()

/home/tim/miniconda3/lib/python3.6/site-packages/matplotlib/__init__.py:942: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

In [37]:
len(months['January']), len(months['February']), len(months['March'])

(467, 2368, 259)

In [38]:
# almost half the total amount of data was from MMIRS in april...
len(fixed)

3094

In [39]:
# now use the between_time() method to split the data into first and second halves of the night.
first_half = fixed.between_time(start_time='00:00', end_time='07:00')
second_half = fixed.between_time(start_time='07:00', end_time='14:00')
#first_half = months['December'].between_time(start_time='00:00', end_time='07:00')
#second_half = months['December'].between_time(start_time='07:00', end_time='14:00')
med_1st = np.median(first_half['seeing'])
med_2nd = np.median(second_half['seeing'])

In [40]:
# plot the results and show that the seeing is better later in the night...
plt.hist(first_half['seeing'], bins=100, range=(0.0, 4.0), label="1st", alpha=0.6)
plt.hist(second_half['seeing'], bins=100, range=(0.0, 4.0), label="2nd", alpha=0.6)
plt.legend(["1st Half: %.2f\"" % med_1st, "2nd Half: %.2f\"" % med_2nd])
plt.xlabel("Seeing (arcsec)")
plt.ylabel("N")
#plt.title("Data and median seeing values grouped by half of the night")
plt.savefig("2018q1_1st2nd.pdf")
plt.show()

/home/tim/miniconda3/lib/python3.6/site-packages/matplotlib/__init__.py:942: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()

In [41]:
# make a new data frame that resamples the WFS data daily into the median, min, and max seeing for each day.
fig, ax = plt.subplots()
daily = pd.DataFrame()
daily['seeing'] = fixed.seeing.resample('D').median()
daily['max'] = fixed.seeing.resample('D').max()
daily['min'] = fixed.seeing.resample('D').min()
lowb = daily['seeing'] - daily['min']
upb = daily['max'] - daily['seeing']
plt.errorbar(daily.index, daily['seeing'], yerr=[lowb, upb], fmt='o')
labels = ax.get_xticklabels()
plt.setp(labels, rotation=20, fontsize=8, ha='right')
plt.ylabel("Seeing (arcsec)")
#plt.title("Nightly median seeing")
plt.savefig("2018q14_nightly.pdf")
plt.show()

/home/tim/miniconda3/lib/python3.6/site-packages/matplotlib/__init__.py:942: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


FigureCanvasNbAgg()